# Retail Restock Forecasting Pipeline
This notebook simulates sales, forecasts demand, queries Gemini AI, decides restock, and uploads results to Google Cloud Storage.

## 0️⃣ Install dependencies (if needed)

In [ ]:
# !pip install --quiet pandas numpy statsmodels google-cloud-storage google-cloud-aiplatform

## 1️⃣ Imports

In [ ]:
import time, json, re, math, random
from datetime import datetime
import numpy as np, pandas as pd
try:
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
    HAS_HW = True
except Exception:
    HAS_HW = False
from google.colab import auth
auth.authenticate_user()
from google.cloud import storage
import vertexai
from vertexai.preview import generative_models

## 2️⃣ Configuration

In [ ]:
PROJECT_ID = "augmented-axe-476116-h6"
BUCKET_NAME = "retail-agent-data"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)
gemini = generative_models.GenerativeModel("projects/augmented-axe-476116-h6/locations/us-central1/publishers/google/models/gemini-2.5-flash")
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

## 3️⃣ Helper Functions

In [ ]:
# Safe Gemini call with retries
def _safe_gemini_text(prompt, max_retries=3, backoff=2):
    last_exc = None
    for attempt in range(max_retries):
        try:
            resp = gemini.generate_content(prompt)
            text = getattr(resp, 'text', None)
            if text is None:
                text = str(resp)
            return text.strip()
        except Exception as e:
            last_exc = e
            time.sleep(backoff ** attempt)
    raise last_exc

# Explain forecast using Gemini
def explain_forecast(item, remaining, predicted_demand, initial_stock, predicted_stockout_days, restock_threshold=0.25):
    prompt = f"""
You are an inventory decision assistant. Return ONLY a single JSON object (no additional prose)
with these keys: restock (true/false), restock_qty (integer), reason (1-2 sentences), confidence (0.0-1.0).

Facts:
- item: {item}
- initial_stock: {initial_stock}
- remaining_stock: {remaining}
- predicted_demand_next_5_days: {predicted_demand:.2f}
- predicted_stockout_in_days: {predicted_stockout_days}

Rules (internal):
1) Recommend restock if remaining_stock < initial_stock * {restock_threshold} OR remaining_stock < predicted_demand_next_5_days.
2) Suggest restock_qty = max(10, round(predicted_demand_next_5_days - remaining_stock)) when restock is true.
3) If unsure, set confidence lower (0.2-0.5).
"""
    try:
        text = _safe_gemini_text(prompt)
        m = re.search(r'\{.*\}', text, re.S)
        parsed = json.loads(m.group()) if m else json.loads(text)
        parsed['restock'] = bool(parsed.get('restock', False))
        parsed['restock_qty'] = int(parsed.get('restock_qty', 0))
        parsed['reason'] = str(parsed.get('reason','')).strip()
        parsed['confidence'] = float(parsed.get('confidence',0.0))
        return parsed
    except Exception:
        rule_restock = (remaining < initial_stock * restock_threshold) or (remaining < predicted_demand)
        rule_qty = max(10, int(max(0, math.ceil(predicted_demand - remaining)))) if rule_restock else 0
        return {'restock': rule_restock, 'restock_qty': rule_qty, 'reason': f'(Fallback rule) remaining={remaining}, predicted_demand={predicted_demand:.1f}', 'confidence': 0.0}

# Forecast helper
def forecast_series(series, days_ahead=5):
    if HAS_HW and len(series) >= 4:
        try:
            model = ExponentialSmoothing(series.astype(float), trend='add', seasonal=None, initialization_method='estimated')
            fit = model.fit(optimized=True)
            return [float(x) for x in fit.forecast(days_ahead)]
        except:
            pass
    avg = float(series.tail(7).mean()) if len(series) >= 3 else float(series.mean() if len(series)>0 else 0.0)
    return [max(0.0, avg)]*days_ahead

## 4️⃣ Simulate sales data

In [ ]:
items = {
    'Milk001': {'initial_stock': 300, 'daily_sales_avg': 15},
    'Bread002': {'initial_stock': 200, 'daily_sales_avg': 10},
    'Eggs003':  {'initial_stock': 50,  'daily_sales_avg': 25},
    'Rice005':  {'initial_stock': 400, 'daily_sales_avg': 12},
    'Oil013':   {'initial_stock': 250, 'daily_sales_avg': 8},
    'Fish014':  {'initial_stock': 18,  'daily_sales_avg': 9},
    'Sugar020': {'initial_stock': 350, 'daily_sales_avg': 14},
    'Salt021':  {'initial_stock': 100, 'daily_sales_avg': 6},
    'Apples022':{'initial_stock': 240, 'daily_sales_avg': 12},
    'Cheese023':{'initial_stock': 150, 'daily_sales_avg': 9}
}
np.random.seed(42)
days = pd.date_range('2025-09-20', periods=15, freq='D')
sales_records = []
for item, data in items.items():
    avg = data['daily_sales_avg']
    sales = np.maximum(np.random.normal(avg, avg*0.2, size=15).astype(int), 0)
    for i,d in enumerate(days):
        sales_records.append({'date': d.strftime('%Y-%m-%d'), 'item_id': item, 'sales': int(sales[i])})
df_sales = pd.DataFrame(sales_records)

## 5️⃣ Forecast & Build Restock Requests

In [ ]:
restock_threshold = 0.25
restock_requests = []
for item_id in df_sales['item_id'].unique():
    series = df_sales[df_sales['item_id']==item_id]['sales']
    forecast = forecast_series(series,5)
    predicted_avg = float(np.mean(forecast))
    predicted_demand_5 = predicted_avg*5
    initial_stock = items[item_id]['initial_stock']
    remaining_stock = initial_stock - series.sum()
    predicted_stockout_days = int(round(remaining_stock/(predicted_avg+1e-9))) if predicted_avg>0 else 9999
    rule_restock = (remaining_stock<initial_stock*restock_threshold) or (remaining_stock<predicted_demand_5)
    rule_qty = max(0,int(math.ceil(predicted_demand_5-remaining_stock))) if rule_restock else 0
    ai = explain_forecast(item_id, remaining_stock, predicted_demand_5, initial_stock, predicted_stockout_days, restock_threshold)
    final_restock = bool(rule_restock or ai.get('restock',False))
    final_qty = int(ai.get('restock_qty',0)) if ai.get('restock_qty',0)>0 else rule_qty
    restock_requests.append({
        'item_id':item_id,
        'location':[int(np.random.randint(1,10)),int(np.random.randint(1,10))],
        'urgency': round(min(1.0,1-max(0,remaining_stock)/(initial_stock+1e-6)),2),
        'quantity': int(final_qty),
        'predicted_stockout_in_days': int(max(0,predicted_stockout_days)),
        'remaining_stock': int(remaining_stock),
        'predicted_demand_next_5_days': round(predicted_demand_5,2),
        'rule_based': {'restock':rule_restock,'restock_qty':rule_qty},
        'ai_recommendation': ai,
        'final_restock_decision': final_restock
    })

## 6️⃣ Save & Upload Results

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'restock_requests_{timestamp}.json'
with open(output_file,'w') as f:
    json.dump(restock_requests,f,indent=2)
blob = bucket.blob(f'restock_requests/{output_file}')
blob.upload_from_filename(output_file)
print(f'✅ Uploaded → gs://{BUCKET_NAME}/restock_requests/{output_file}')
df_out = pd.DataFrame(restock_requests)[['item_id','remaining_stock','predicted_demand_next_5_days','quantity','predicted_stockout_in_days','final_restock_decision']]
print('\n--- Summary table ---')
print(df_out.to_string(index=False))